# KaBOB-MOPs

## KaBOB Interface Test

KaBOBInterface is used to mopify and parse KaBOB triples. OpenKaBOB opens a connection to KaBOB and closes it using a `with` statement

In [1]:
from KaBOB_Interface import KaBOBInterface
import KaBOB_Constants
import networkx as nx
import pickle

In [1]:
with OpenKaBOB("KaBOB_credentials.txt") as kabob:
    print('Triples in KaBOB:', kabob.size())

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:OpenKaBOB:Closed kabob_prod_feb2018


Triples in KaBOB: 283830446


### Basic Querying

#### Without using the interface (using just agraph_python)

In [4]:
from KaBOB_Interface import OpenKaBOB

p53 = kabob.createURI("http://purl.obolibrary.org/obo/PR_P04637")
label = kabob.createURI("http://www.w3.org/2000/01/rdf-schema#label")

with OpenKaBOB() as kabob:
    print('Triples in KaBOB:', kabob.size())
    
    print('Label for p53')
    p53_labels = kabob.getStatements(p53, label).asList()
    p53_label = p53_labels[0].getObject()
    print("\t%s" % p53_label)
    
    print()
    
    print('Triples with p53 as the subject')
    p53_as_subject = kabob.getStatements(p53, None, None)
    with p53_as_subject:
        p53_as_subject.enableDuplicateFilter()  # This may or may not be necessary all the time
        for statement in p53_as_subject:
            object_labels = kabob.getStatements(statement.getObject(), label).asList()
            
            if object_labels:
                print("\t%s %s %s" % (p53_label, statement.getPredicate().getLocalName(), object_labels[0].getObject()))
    
    print()
    
    print('Triples with p53 as the object')
    p53_as_object = kabob.getStatements(None, None, p53)
    with p53_as_object:
        p53_as_object.enableDuplicateFilter()
        for statement in p53_as_object:
            subject_labels = kabob.getStatements(statement.getSubject(), label).asList()
            
            if subject_labels:
                print("\t%s %s %s" % (p53_label, statement.getPredicate().getLocalName(), subject_labels[0].getObject()))


DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Triples in KaBOB: 283830446
Label for p53
	"cellular tumor antigen p53 (human)"

Triples with p53 as the subject
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53"
	"cellular tumor antigen p53 (human)" subClassOf "Homo sapiens protein"

Triples with p53 as the object
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 isoform 1 (human)"
	"cellular tumor antigen p53 (human)" annotatedSource "PR:P04637 located_in GO:0005654"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 15 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 14 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 dimethylated 3 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 16 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 1 (human)"
	"cel

DEBUG:OpenKaBOB:Closed kabob_prod_feb2018



	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 ubiquitinated form (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 18 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 4 (human)"


#### With the interface

In [2]:
from KaBOB_Interface import KaBOBInterface

p53 = "obo:PR_P04637"

with KaBOBInterface("KaBOB_credentials.txt") as interface:
    print('Label for p53')
    p53_label = interface.node_print_name(p53)
    print("\t%s" % p53_label)
    
    print()
    
    print('Triples with p53 as the subject')
    for statement in interface.agraph_get_statements(s=p53):
        object_label = interface.node_print_name(statement.getObject())
        predicate_label = interface.node_print_name(statement.getPredicate())
        print("\t%s %s %s" % (p53_label, predicate_label, object_label))
    
    print()
    
    print('Triples with p53 as the object')
    for statement in interface.agraph_get_statements(o=p53):
        subject_label = interface.node_print_name(statement.getSubject())
        predicate_label = interface.node_print_name(statement.getPredicate())
        print("\t%s %s %s" % (subject_label, predicate_label, p53_label))

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Label for p53
	cellular tumor antigen p53 (human)

Triples with p53 as the subject
	cellular tumor antigen p53 (human) database_cross_reference "UniProtKB:P04637"
	cellular tumor antigen p53 (human) database_cross_reference "Reactome:R-HSA-69488"
	cellular tumor antigen p53 (human) id "PR:P04637"
	cellular tumor antigen p53 (human) has_obo_namespace "protein"
	cellular tumor antigen p53 (human) definition "A cellular tumor antigen p53 that is encoded in the genome of human."
	cellular tumor antigen p53 (human) label "cellular tumor antigen p53 (human)"
	cellular tumor antigen p53 (human) comment "Category=organism-gene."
	cellular tumor antigen p53 (human) subClassOf cellular tumor antigen p53
	cellular tumor antigen p53 (human) subClassOf Homo sapiens protein
	cellular tumor antigen p53 (human) subClassOf pr_genid1776552
	cellular tumor antigen p53 (human) subClassOf pr_genid338241
	cellular tumor antigen p53 (human) equivalentClass pr_genid338237
	cellular tumor antigen p53 (human) t

DEBUG:OpenKaBOB:Closed kabob_prod_feb2018


	cellular tumor antigen p53 phosphorylated 4 (human) subClassOf cellular tumor antigen p53 (human)


### Mopification

In [1]:
p53 = "obo:PR_P04637"

with KaBOBInterface("KaBOB_credentials.txt") as interface:
    bio_p53 = interface.bio(interface.create_uri(p53))
    interface.mopify(bio_p53)
        
#     interface.draw(layout=nx.circular_layout)

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:KaBOBInterface:> cellular tumor antigen p53 (human)
DEBUG:KaBOBInterface:	> cellular tumor antigen p53
DEBUG:KaBOBInterface:		> protein
DEBUG:KaBOBInterface:			> biopolymer
DEBUG:KaBOBInterface:				> interactor type
DEBUG:KaBOBInterface:				< interactor type
DEBUG:KaBOBInterface:			< biopolymer
DEBUG:KaBOBInterface:			> amino acid chain
DEBUG:KaBOBInterface:				> molecular entity
DEBUG:KaBOBInterface:					> chemical entity
DEBUG:KaBOBInterface:						> object
DEBUG:KaBOBInterface:							> material entity
DEBUG:KaBOBInterface:								> independent continuant
DEBUG:KaBOBInterface:									> continuant
DEBUG:KaBOBInterface:										> entity
DEBUG:KaBOBInterface:										< entity
DEBUG:KaBOBInterface:										> occurrent
DEBUG:KaBOBInterface:										< occurrent
DEBUG:KaBOBInterface:									< continuant
DEBUG:KaBOBInterface:									> specifically dependent continuant
DEBUG:KaBOBInterfac

DEBUG:KaBOBInterface:																														< kingdom
DEBUG:KaBOBInterface:																													< Fungi
DEBUG:KaBOBInterface:																												< Metazoa
DEBUG:KaBOBInterface:																											< Eumetazoa
DEBUG:KaBOBInterface:																											> Cnidaria
DEBUG:KaBOBInterface:																												> phylum
DEBUG:KaBOBInterface:																												< phylum
DEBUG:KaBOBInterface:																											< Cnidaria
DEBUG:KaBOBInterface:																										< Bilateria
DEBUG:KaBOBInterface:																										> Platyhelminthes
DEBUG:KaBOBInterface:																										< Platyhelminthes
DEBUG:KaBOBInterface:																									< Deuterostomia
DEBUG:KaBOBInterface:																								< Chordata
DEBUG:KaBOBInterface:																								> subphylum
DEBUG:KaBOBInterface:																								< subphylum
DEBUG:KaBOBInterface:																							< Craniata <chordata>
DEBUG:KaBOBInterface:																				

In [3]:
with KaBOBInterface("KaBOB_credentials.txt") as interface:
    
    print("Reading mops")
    mops = pickle.load(open("E:/Documents/KaBOB/pickles/mops.pickle", "rb"))
    interface.mops = mops
    
    print("Calculating statistics")
    frame_stats, role_stats = interface.mops.get_statistics()
    
    print("***Frame Statistics***")
    for key, value in frame_stats.items():
        print("\tNumber of %s: %d" % (key, value))
    
    print()
    
    print("***Role Statistics***")
    for key, value in role_stats.items():
        print("\tNumber of %s: %d" % (key, value))
        
    

DEBUG:KaBOBInterface:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Reading mops


DEBUG:KaBOBInterface:Closed AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Calculating statistics
***Frame Statistics***
	Number of mop: 31211

***Role Statistics***
	Number of only_in_taxon: 3975
	Number of equivalent: 674
	Number of has_participant: 283
	Number of causes or contributes to condition: 172
	Number of transports or maintains localization of: 93
	Number of has target end location: 93
	Number of never_in_taxon: 25


In [23]:
hyper_cholesterol = "obo:GO_0034383"

mops = None
with KaBOBInterface("KaBOB_credentials.txt") as interface:
    bio_hyper_cholesterol = interface.get_bio(hyper_cholesterol)
    
    hyper_cholesterol_statements_of_interest = interface.get_statements(s=bio_hyper_cholesterol)
    hyper_cholesterol_statements_of_interest.extend(interface.get_statements(o=bio_hyper_cholesterol))
    
    print("Hyper cholesterol is involved in %d statements of interest" % len(hyper_cholesterol_statements_of_interest))
    
    count = 0
    for statement in hyper_cholesterol_statements_of_interest:
        print("****** %d ******" % count)
        interface.mopify(statement.getSubject())
        count += 1
        
    mops = interface.mops

mops.draw_mops("E:/GDrive/Projects/Biology/images")

DEBUG:KaBOBInterface:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:KaBOBInterface:> low-density lipoprotein particle clearance


Hyper cholesterol is involved in 28 statements of interest
****** 0 ******


DEBUG:KaBOBInterface:	> plasma lipoprotein particle clearance
DEBUG:KaBOBInterface:		> multicellular organismal process
DEBUG:KaBOBInterface:			> biological_process
DEBUG:KaBOBInterface:				> processual entity
DEBUG:KaBOBInterface:					> occurrent
DEBUG:KaBOBInterface:						> entity
DEBUG:KaBOBInterface:						< entity
DEBUG:KaBOBInterface:					< occurrent
DEBUG:KaBOBInterface:				< processual entity
DEBUG:KaBOBInterface:			< biological_process
DEBUG:KaBOBInterface:			> Saccharomyces cerevisiae
DEBUG:KaBOBInterface:				> Saccharomyces
DEBUG:KaBOBInterface:					> Saccharomycetaceae
DEBUG:KaBOBInterface:						> Saccharomycetales
DEBUG:KaBOBInterface:							> Saccharomycetes
DEBUG:KaBOBInterface:								> Saccharomycotina
DEBUG:KaBOBInterface:									> saccharomyceta
DEBUG:KaBOBInterface:										> Ascomycota
DEBUG:KaBOBInterface:											> Dikarya
DEBUG:KaBOBInterface:												> Fungi
DEBUG:KaBOBInterface:													> Opisthokonta
DEBUG:KaBOBInterface:														> Eukaryota
DE

****** 1 ******
****** 2 ******


DEBUG:KaBOBInterface:> GO_0034383
DEBUG:KaBOBInterface:< GO_0034383


...

****** 26 ******


DEBUG:KaBOBInterface:	> B_Qh1fB84OVMqwAL7kxdgi5G3_j6s
DEBUG:KaBOBInterface:		> clathrin heavy chain 1 (human)
DEBUG:KaBOBInterface:			> GP_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:				> GGP_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:				< GGP_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:				> GPGPV_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:					> GGPV_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:					< GGPV_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:				< GPGPV_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:			< GP_B_-Fplro3EK-_n-9u8M02SO8Y6Ks0
DEBUG:KaBOBInterface:			> clathrin heavy chain 1
DEBUG:KaBOBInterface:			< clathrin heavy chain 1
DEBUG:KaBOBInterface:		< clathrin heavy chain 1 (human)
DEBUG:KaBOBInterface:	< B_Qh1fB84OVMqwAL7kxdgi5G3_j6s
DEBUG:KaBOBInterface:> B_xVTrMuZOOnagoBINTYUdXCPs1j0
DEBUG:KaBOBInterface:< B_xVTrMuZOOnagoBINTYUdXCPs1j0


****** 27 ******


DEBUG:KaBOBInterface:	> B_fHl68X3dA9__qVWtks1Zt5i93i4
DEBUG:KaBOBInterface:		> lysosomal acid lipase/cholesteryl ester hydrolase (human)
DEBUG:KaBOBInterface:			> GP_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:				> GPGPV_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:					> GGPV_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:					< GGPV_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:				< GPGPV_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:				> GGP_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:				< GGP_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:			< GP_B_Gl9TkDGmlHmevRs9snXaNIZtccE
DEBUG:KaBOBInterface:			> lysosomal acid lipase/cholesteryl ester hydrolase
DEBUG:KaBOBInterface:			< lysosomal acid lipase/cholesteryl ester hydrolase
DEBUG:KaBOBInterface:		< lysosomal acid lipase/cholesteryl ester hydrolase (human)
DEBUG:KaBOBInterface:	< B_fHl68X3dA9__qVWtks1Zt5i93i4
DEBUG:KaBOBInterface:> B_t3iaKx0H4bdSFdmAcGcpfgRFsuQ
DEBUG:KaBOBInterface:< B_t3ia

Drawing graphs


In [16]:
print("Calculating statistics")
frame_stats, role_stats = mops.get_statistics()

print("***Frame Statistics***")
for key, value in frame_stats.items():
    print("\tNumber of %s: %d" % (key, value))

print()

print("***Role Statistics***")
for key, value in role_stats.items():
    print("\tNumber of %s: %d" % (key, value))

Calculating statistics
***Frame Statistics***
	Number of mop: 281

***Role Statistics***
	Number of never_in_taxon: 2
	Number of only_in_taxon: 24
	Number of equivalent: 1
	Number of has_participant: 24


In [35]:
with KaBOBInterface("KaBOB_credentials.txt") as interface:
    print(interface.get_node_label(bio_hyper_cholesterol, interface.get_labels(bio_hyper_cholesterol)))

DEBUG:KaBOBInterface:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:KaBOBInterface:Closed AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


low-density lipoprotein particle clearance


Here, we collapse the children of LDL clearance

In [122]:
from MOPs import MOPs

neighbor_of_bio_hyper_cholesterol = lambda u, v: (u == bio_hyper_cholesterol and v in mops.neighbors(u))
def label_edges(B, C):
    for u in B:
        for v in C:
            if (u,v) in  mops.edges:
                return {"label": mops.edges[u, v]["label"]}

def label_nodes(B): 
    label = None
    for u in B:
         if u in nx.get_node_attributes(mops, "label"):
            label ={"label": nx.get_node_attributes(mops, "label")[u]}
            if u == bio_hyper_cholesterol:
                return label
            
    return label
            

collapsed_graph = nx.quotient_graph(mops, neighbor_of_bio_hyper_cholesterol, edge_data=label_edges, node_data=label_nodes)

collapsed_mops = MOPs()
collapsed_mops.add_nodes_from(collapsed_graph.nodes(data=True))
collapsed_mops.add_edges_from(collapsed_graph.edges)
collapsed_mops.draw_mops("E:/GDrive/Projects/Biology/images")

In [104]:
collapsed_graph.number_of_nodes()

280

In [135]:
nx.get_edge_attributes(collapsed_graph, "label")

{frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_gHRGYnSlCSJ09CLanxc_Cysl9n8>}): 'APOB_HUMAN',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_WgGGsNipZjBCX-gYx4rFLm7GARM>}): 'Saccharomyces cerevisiae',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_J_hJJTg_QOKKIvJt6AoJ_C_t8AU>}): 'Homo sapiens protein',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_r2rur2b3Iw88n6W2QrSsLu3bzf4>}): 'B_r2rur2b3Iw88n6W2QrSsLu3bzf4',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/GGPV_B_JcjyO1U5QsSI9AOCcr4dpo35KeQ>}): 'GGPV_B_JcjyO1U5QsSI9AOCcr4dpo35KeQ',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_LJHU9pBpgkTn02i1BDTuGZHvekI>}): 'Homo sapiens',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_HefLS9fnex5-hJkdhjmAl7Gk-YM>}): 'Ascomycota',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_vozEr9-hmp8LLzU8pIyZXAUEeNY>}): 'Opisthokonta',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/B_82ukdgYlRN_R6FjmfiZWRy9VSHA>}): 'biological_process',
 frozenset({<http://ccp.ucdenver.edu/kabob/bio/GPGPV_B_WdYy3z